In [1]:
import os, sys
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import darts
from darts.dataprocessing.transformers.boxcox import BoxCox
from darts.models import LightGBMModel, XGBModel, LinearRegressionModel, TFTModel, NHiTSModel, RNNModel, TFTModel
from darts.metrics import smape, mape, mase, mse, rmse, r2_score, mae
from darts.dataprocessing.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler   
from darts.dataprocessing.transformers.scaler import Scaler
from darts.utils.missing_values import extract_subseries

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
from wandb.xgboost import WandbCallback

import wandb
wandb.login()


import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
repo_name = 'net-load-forecasting'
assert os.getcwd()[-len(repo_name):] == "net-load-forecasting", "Working directory is not the git repo root directory"


from utils.utils import *

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\net-load-forecasting


In [2]:
clean_data_path = os.path.join(os.getcwd(),'data','clean_data')
model_data_path = os.path.join(os.getcwd(),'data','model_data')

In [4]:
# run parameters

config_dataset = {
    'spatial_scale': '1_county',
    'METER': '2',
    'META': '1',
}

config_dataset['temp_resolution'] = 1 if config_dataset['METER'] == '1' else 15 # in minutes

config_modeldesign = {'boxcox': False,
                    'horizon_in_hours': 24, # in hours
                    'lookback_in_hours': 36, # in hours
                    'liklihood': None,
                    'holiday': True,
                    'datetime_encodings': False,
                    }
                   

if config_dataset['temp_resolution'] == 1:
     timestep_encoding = timestep_encoding = ["hour", "minute"]
elif config_dataset['temp_resolution'] == 15:
     timestep_encoding = ['quarter']


config_encoders =  {
                    "cyclic": {"past": timestep_encoding}, 
                    "position": {"past": ["relative",]},
                    "datetime_attribute": {"past": ["dayofweek", "week"]},
                    'position': {'past': ['relative']},
            }



In [5]:
# calculate derived parameters
datetime_encoders = config_encoders if config_modeldesign['datetime_encodings'] else None
timesteps_per_hour = int(60 / config_dataset['temp_resolution'])
n_lags = config_modeldesign['lookback_in_hours'] * timesteps_per_hour
n_ahead = config_modeldesign['horizon_in_hours'] * timesteps_per_hour
list_metrics = [smape, mape, rmse, r2_score, mae] # evaluation metrics
eval_stride = int(np.sqrt(n_ahead)) # evaluation stride, how often to evaluate the model, in this case we evaluate every n_ahead steps

df = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key=f"{config_dataset['temp_resolution']}min/netload")

df_pv = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key=f"{config_dataset['temp_resolution']}min/community_pv")

df_meta = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key='pv_metadata')

df_irr = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key=f"{config_dataset['temp_resolution']}min/weather")
df_irr.rename({'temperature': 'temp_air'}, axis=1, inplace=True)

df_pv_forecast = pd.read_hdf(os.path.join(model_data_path, "pv_model_results.h5"), key=f"{config_dataset['temp_resolution']}min/pv_forecast_META-{config_dataset['META']}")


In [24]:
train_end = '2015-10-01'
val_end = '2016-04-01'

df_train_int = df.loc[:train_end]
df_val_int = df.loc[train_end:val_end]
df_test_int = df.loc[val_end:]

df_cov_dir_train = df_irr.loc[:train_end]
df_cov_dir_val = df_irr.loc[train_end:val_end]
df_cov_dir_test = df_irr.loc[val_end:]

df_cov_pv_train = df_pv_forecast.loc[:train_end]
df_cov_pv_val = df_pv_forecast.loc[train_end:val_end]
df_cov_pv_test = df_pv_forecast.loc[val_end:]

df_train_add = df_train_int + df_cov_pv_train.values
df_train_add[df_train_add < 0] = 0
df_val_add = df_val_int + df_cov_pv_val.values
df_val_add[df_val_add < 0] = 0
df_test_add = df_test_int + df_cov_pv_test.values
df_test_add[df_test_add < 0] = 0



In [29]:
# In this study we are comparing three different model setups: integrated, additive and direct for net load forecasting
model_setups = {'integrated': {'target': (df_train_int, df_val_int, df_test_int), 'covs': (df_cov_pv_train, df_cov_pv_val, df_cov_pv_test)},
                'additive': {'target': (df_train_add, df_val_add, df_test_add), 'covs': (df_cov_dir_train, df_cov_dir_val, df_cov_dir_test)},
                'direct': {'target': (df_train_int, df_val_int, df_test_int), 'covs': (df_cov_dir_train, df_cov_dir_val, df_cov_dir_test)}}


In [56]:
predictions_per_model = {}

for model_setup, data in model_setups.items():

    df_train, df_val, df_test = data['target']
    df_cov_train, df_cov_val, df_cov_test = data['covs']

    # Into Darts format
    ts_train = darts.TimeSeries.from_dataframe(df_train, freq=str(config_dataset['temp_resolution']) + 'min')
    ts_train = extract_subseries(ts_train)
    ts_val = darts.TimeSeries.from_dataframe(df_val, freq=str(config_dataset['temp_resolution']) + 'min')
    ts_val = extract_subseries(ts_val)
    ts_test = darts.TimeSeries.from_dataframe(df_test, freq=str(config_dataset['temp_resolution']) + 'min')
    ts_test = extract_subseries(ts_test)

    # Covariates
    ts_cov_train = darts.TimeSeries.from_dataframe(df_cov_train, freq=str(config_dataset['temp_resolution']) + 'min')
    ts_cov_val = darts.TimeSeries.from_dataframe(df_cov_val, freq=str(config_dataset['temp_resolution']) + 'min')
    ts_cov_test = darts.TimeSeries.from_dataframe(df_cov_test, freq=str(config_dataset['temp_resolution']) + 'min')


    # Reviewing subseries to make sure they are long enough
    ts_train, ts_cov_train = review_subseries(ts_train, (n_lags +n_ahead), ts_cov_train)
    ts_val, ts_cov_val = review_subseries(ts_val,  (n_lags +n_ahead), ts_cov_val)
    ts_test, ts_cov_test = review_subseries(ts_test,  (n_lags +n_ahead), ts_cov_test)

    # getting the index of the longest subseries, to be used for evaluation later
    longest_ts_val_idx = get_longest_subseries_idx(ts_val)
    longest_ts_test_idx = get_longest_subseries_idx(ts_test)


    # Load pipeline
    pipeline = Pipeline( # missing values have been filled in the 'data_prep.ipynb'
                        [
                        #BoxCox() if config_modeldesign['boxcox'] else None,
                        Scaler(MinMaxScaler()),
                        ]
                        )

    ts_train_piped = pipeline.fit_transform(ts_train)
    ts_val_piped = pipeline.transform(ts_val)
    ts_test_piped = pipeline.transform(ts_test)

    # Future Covariate Pipeline

    pipeline_weather = Pipeline([Scaler(MinMaxScaler())])
    ts_cov_train_piped = pipeline_weather.fit_transform(ts_cov_train)
    ts_cov_val_piped = pipeline_weather.transform(ts_cov_val)
    ts_cov_test_pipied = pipeline_weather.transform(ts_cov_test)


    trg_train_inversed = pipeline.inverse_transform(ts_train_piped, partial=True) # inverse transform the target, we need the original values for the evaluation
    trg_val_inversed = pipeline.inverse_transform(ts_val_piped, partial=True)[longest_ts_val_idx] # inverse transform the target, we need the original values for the evaluation
    trg_test_inversed = pipeline.inverse_transform(ts_test_piped, partial=True)[longest_ts_test_idx] # inverse transform the target, we need the original values for the evaluation


    model = LinearRegressionModel(lags=48,
                    lags_future_covariates=[0],
                    add_encoders=datetime_encoders, 
                    output_chunk_length=n_ahead, 
                    likelihood=None,
                    random_state=42
                    )

    print("Training model...")
    model.fit(ts_train_piped, future_covariates = ts_cov_train)

    print("Evaluating model...")
    predictions, score = predict_testset(model, 
                                    ts_test_piped[longest_ts_test_idx], 
                                    ts_cov_test[longest_ts_test_idx],
                                    n_lags, n_ahead, eval_stride, pipeline,
                                    )
    
    predictions.columns = ['prediction'+model_setup]

    # subtracting the covariates from the predictions, since we are predicting the net load
    if model_setup == 'additive':
        predictions -= df_cov_pv_test.reindex(predictions.index).values

    predictions_per_model[model_setup] = predictions



Training model...
Evaluating model...
Score:  0.16173019000877464
Training model...
Evaluating model...
Score:  0.12640151077009518
Training model...
Evaluating model...
Score:  0.17072687886199972


In [57]:
df_predictions = pd.concat(predictions_per_model.values(), axis=1)

In [58]:

print("Plotting predictions...")
df_compare = pd.concat([trg_test_inversed.pd_dataframe(), df_predictions], axis=1).dropna()

px.line(df_compare)

Plotting predictions...
